In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

import sys
sys.path.append('..')
from my_workspace.config import *

### Retrieve all sku and transaction information from the join Black Friday table

In [2]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')
query = "SELECT * FROM group_2.joined_trnsact"
df = pd.read_sql_query(query, engine)

In [3]:
df.shape

(220732, 13)

There are 220,732 sku records, containing purchased and returned items.

In [4]:
df.stype.value_counts() 

stype
P    212350
R      8382
Name: count, dtype: int64

In [5]:
#filter the purchase category because we want to find items people want to buy
df[df["stype"] == "P"]

,sku,cost,retail,stype,quantity,orgprice,amt,style,color,size,packsize,vendor,brand
0,8936935,9.58,6.00,P,1,24.0,24.00,35-92 413641,WHEAT NUBU,L,1,914938,KIDS HEA
1,8940963,4.31,2.75,P,1,11.0,8.25,005 U61516,004GREY HE,L,3,4012768,TOMMY HI
2,8944045,28.05,17.00,P,1,68.0,29.99,CC412,MULTI,M,1,8043525,KORET OF
3,8944045,25.00,14.50,P,1,58.0,29.00,CC412,MULTI,M,1,8043525,KORET OF
4,8972921,14.72,17.50,P,1,35.0,35.00,T-PIL TRA-QL,TERRA,BOU PIL,2,619402,STATUS Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220727,8902921,87.66,99.50,P,1,199.0,199.00,COM TRA-K-,TERRA,K COMF,1,619402,STATUS Q
220728,8902921,87.66,99.50,P,1,199.0,199.00,COM TRA-K-,TERRA,K COMF,1,619402,STATUS Q
220729,8902921,87.66,99.50,P,1,199.0,199.00,COM TRA-K-,TERRA,K COMF,1,619402,STATUS Q
220730,8902921,87.66,99.50,P,1,199.0,199.00,COM TRA-K-,TERRA,K COMF,1,619402,STATUS Q


In [6]:
df.info() #there is no null vaue no nothing needs to be done in this table 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220732 entries, 0 to 220731
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   sku       220732 non-null  object 
 1   cost      220732 non-null  float64
 2   retail    220732 non-null  float64
 3   stype     220732 non-null  object 
 4   quantity  220732 non-null  int64  
 5   orgprice  220732 non-null  float64
 6   amt       220732 non-null  float64
 7   style     220732 non-null  object 
 8   color     220732 non-null  object 
 9   size      220728 non-null  object 
 10  packsize  220732 non-null  int64  
 11  vendor    220732 non-null  object 
 12  brand     220280 non-null  object 
dtypes: float64(4), int64(2), object(7)
memory usage: 21.9+ MB


In [7]:
df.describe()

,cost,retail,quantity,orgprice,amt,packsize
count,220732.000000,220732.000000,220732.0,220732.000000,220732.000000,220732.000000
mean,13.759040,21.422476,1.0,29.484426,25.341237,2.465959
std,14.338254,25.592317,0.0,33.478250,26.020788,6.519093
min,0.000000,0.000000,1.0,0.000000,0.010000,1.000000
25%,6.000000,7.000000,1.0,13.500000,11.000000,1.000000
50%,10.250000,16.000000,1.0,20.000000,19.000000,1.000000
75%,17.250000,25.000000,1.0,37.500000,32.000000,3.000000
max,325.000000,795.000000,1.0,795.000000,695.000000,801.000000


In [8]:
# explore the only 2 column that has quantitative meanings 
df.quantity.describe()

count    220732.0
mean          1.0
std           0.0
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           1.0
Name: quantity, dtype: float64

Since the feature `quantity` is constant (always 1), so it will not help in predicting whether the item will be purchased or not.

In [9]:
df.orgprice.describe()

count    220732.000000
mean         29.484426
std          33.478250
min           0.000000
25%          13.500000
50%          20.000000
75%          37.500000
max         795.000000
Name: orgprice, dtype: float64

In [10]:
df.amt.describe()

count    220732.000000
mean         25.341237
std          26.020788
min           0.010000
25%          11.000000
50%          19.000000
75%          32.000000
max         695.000000
Name: amt, dtype: float64

The summary statistics of total amount of charge was slightly lower than item's orgprice.
It is likely costed by:
- Purchased items have discount
- Customers have coupons or membership card to get discounts

In [11]:
df[df["orgprice"] == 0][["amt", "orgprice"]] 

,amt,orgprice
648,27.50,0.0
3818,69.50,0.0
11468,26.99,0.0
16258,34.00,0.0
18798,28.00,0.0
...,...,...
201670,9.00,0.0
205413,5.00,0.0
205414,5.00,0.0
207155,7.00,0.0


It is interesting to see that some items were purchased for amount greater than the original price of 0.

In [12]:
df[df.amt <= df.orgprice].shape

(220496, 13)

In [13]:
df[df.orgprice < df.amt].shape

(236, 13)

But still, due to all the transactions were on the Black Friday, it made sense that the total amount charged to customers was cheaper than the original price because of the discounts.

In [14]:
# save data to pickle for data cleaning later
df.to_pickle('../data/joined_black_friday_trnsact.pkl')